# Create SPECFEM3D Project Directories
1. Define function for making directories
1. Create directories for each event including the special directories for run0001 

### 1. Define function for making directories

In [ ]:
import os

def make_directory(fqdn):
    if os.path.isdir(fqdn):
        print ("The directory %s has already been created" % fqdn)
        return 0
    try:
        os.makedirs(fqdn, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % fqdn)
        print()
        return -1
    else:
        print ("Creation of the directory %s succeeded" % fqdn)
        print()
        return 0
    

### 2. Create directories for each event including the special directories for run0001

In [ ]:
num_events = 3
proj_root_path = '/Users/mcmac/Documents/Work/Bench/Groningen/data_notebooks/SPECFEM3D/project/'

# list of directories need for every event
common_dir_list = ['/DATA',
                   '/OUTPUT_FILES/DATABASES_MPI',
                   '/SEM',
                   '/OBS',
                   '/SYN',
                   '/FILT_OBS',
                   '/FILT_SYN',]

# list of directories only needed for the primary run0001 dir
primary_dir_list = ['/INPUT_GRADIENT',
                    '/INPUT_KERNELS',
                    '/INPUT_MODEL',
                    '/OUTPUT_MODEL',
                    '/OUTPUT_SUM',
                    '/SMOOTH',
                    '/COMBINE',
                    '/topo']

# define the access rights
access_rights = 0o755

#loop over number of events
for e in range(num_events):
    edir = proj_root_path + '/run' + str(e+1).zfill(4)
    dname = edir
    
    # make for each event
    print('Experiment-%d:\n'%(e+1), dname)
    for d in common_dir_list:
        new_dir = dname + d
        err = make_directory(new_dir)
        if err != 0:
            print('Oops. You messed up creating common dirs!')
            assert False
        
    # make dirs needed only in the primary run0001 dir (used for inversion, model-updating, etc.)
    if e == 0:
        for d in primary_dir_list:
            new_dir = dname + d
            err = make_directory(new_dir)
            if err != 0:
                print('Oops. You messed up creating special dirs!')
                assert False


## Finished